In [1]:
import os
import re
import csv 
import sys
import time
import math
import torch
import random
import shutil
import psutil
import string
import logging
import subprocess
import pandas as pd
import concurrent.futures
import ipywidgets as widgets
import multiprocessing as mp

from glob import glob
from typing import Optional, List
from IPython.display import display
from rdkit import Chem, DataStructs
from z_docking_files.utils import *
from openbabel import openbabel, pybel
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor
from rdkit.Chem import AllChem, Descriptors, Draw 

In [2]:
device = check_availability()

GPU availability in own5: 9.00%
CPU availability in own5: 31.50%


<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 1. Load Data </h2>
</div>

In [3]:
folder_name = "brd4_hll"
pdb_file = "brd4.pdb"  
pdbqt_rigid = "brd4_rigid_10_res.pdbqt"
pdbqt_flex = "brd4_flex_10_res.pdbqt"
input_csv = "output.csv"
threshold = -10.97

In [4]:
flex_folder_name = folder_name + "_flexible"
os.makedirs(os.path.join(flex_folder_name), exist_ok=True)

In [5]:
input_smiles = os.path.join(folder_name, input_csv)
df = pd.read_csv(input_smiles)
df = df[df['Affinity'] < threshold]
df.to_csv(os.path.join(flex_folder_name, 'output.csv'), index=False)
display(df.head())
print(df.shape)

,Name,SMILES,Affinity
0,brd4_chemdiv4079,c1(c(c2c(o1)c(c1c(c(cc(=O)o1)c1ccccc1)c2)C)c1c...,-10.977


(1, 3)


<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 2. Copy PDBQT files and Create ligands path </h2>
</div>

In [6]:
copy_pdbqt_files_flex(df, folder_name, flex_folder_name)
create_ligands_path_flex(flex_folder_name)

<div style="background-color:#8E44AD; color:#ECF0F1; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 3. Uni-Dock Flexible Docking </h2>
</div>

In [6]:
protein_rigid = os.path.join(folder_name, pdbqt_rigid)
protein_flex = os.path.join(folder_name, pdbqt_flex)

ligands_path = os.path.join(flex_folder_name, "pipeline_files/unidock_pdbqt_path.txt")

output_result = os.path.join(flex_folder_name, "pipeline_files/2_pdbqt_out_flex")
os.makedirs(output_result, exist_ok=True)

output_logs = os.path.join(flex_folder_name, "pipeline_files/unidock_output.txt")
os.system("touch " + output_logs)
print(f"\033[1m\033[34mCheck Docking Progress... \033[91m${output_logs}\033[0m")
unidock_command = (
    f"unidock "
    f"--flex {protein_flex} "
    f"--receptor {protein_rigid} "
    f"--gpu_batch $(cat {ligands_path}) "
    f"--search_mode detail "
    f"--scoring vina "
    f"--center_x 86.79 "
    f"--center_y 43.89 "
    f"--center_z 65.43 "
    f"--size_x 20.00 "
    f"--size_y 20.00 "
    f"--size_z 20.00 "
    f"--num_modes 10 "
    f"--dir {output_result} "
    f">> {output_logs} 2>&1"
)

os.system(unidock_command)
print(f"\033[1m\033[34mDocking Process completed\033[0m")

Check Docking Progress... $brd4_hll_flexible/pipeline_files/unidock_output.txt
Docking Process completed


In [8]:
!unidock --help_advanced

Uni-Dock v0.1.0

Input:
  --receptor arg                        rigid part of the receptor (PDBQT or 
                                        PDB)
  --flex arg                            flexible side chains, if any (PDBQT or 
                                        PDB)
  --ligand arg                          ligand (PDBQT)
  --ligand_index arg                    file containing paths to ligands (PDBQT
                                        or SDF
  --paired_batch_size arg               If > 0, uses batching for 
                                        one-ligand-one-protein docking, with 
                                        json config in ligand_index following 
                                        paired_batching.schema.json
  --batch arg                           batch ligand (PDBQT)
  --gpu_batch arg                       gpu batch ligand (PDBQT or SDF)
  --scoring arg (=vina)                 scoring function (ad4, vina or vinardo)

Search space (required):
  --maps arg   

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 4. Extract Affinity values and Model1 Content </h2>
</div>

In [8]:
affinity_from_pdbqt_files_flex(flex_folder_name)
extract_model1_flex(flex_folder_name)

Affnity values extracted and saved in folder: brd4_hll_flexible/pipeline_files/1_extract_affinity_from_pdbqt.csv
Extracted Model_1 content and saved in folder: brd4_hll_flexible/pipeline_files/3_pdbqt_out_flex_m1


<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 5. PDBQT to Mol2 and Mol2 to SDF</h2>
</div>

In [9]:
script_path = os.path.join("z_docking_files", "3_pdbqt_to_sdf_flex.sh")

result = subprocess.run(["/bin/bash", script_path, flex_folder_name])
print(result.stderr)

PDBQT to SDF conversion completed and files saved in folder: brd4_hll_flexible/pipeline_files/5_sdf_out
None


<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 6. PoseBusters Filter </h2>
</div>

In [10]:
script_path = os.path.join("z_docking_files", "4_posebusters_filter_flex.sh")
pdb = os.path.join(folder_name, pdb_file)

result = subprocess.run(["/bin/bash", script_path, flex_folder_name, pdb])
print(result.stdout)

Check PoseBusters Progress... brd4_hll_flexible/pipeline_files/2_pb_out.csv
PoseBusters Filtration Completed
None


In [11]:
process_pb_csv_flex(flex_folder_name)
final_output_flex(flex_folder_name, input_smiles, passes=19)

EmptyDataError: No columns to parse from file